In [109]:
import pandas
import numpy as np

df = pandas.read_csv("meta_scan_csvs/uncleaned/post_update/blinds_up_motion_7.csv")
# drop ts column, not useful
df = df.drop('ts', axis=1).copy()
df

,value,counter_name,t_s
0,0.000000e+00,application_prediction_milliseconds,0.000000
1,0.000000e+00,application_prediction_milliseconds,0.000000
2,0.000000e+00,application_prediction_milliseconds,0.000000
3,0.000000e+00,application_prediction_milliseconds,0.000000
4,0.000000e+00,application_prediction_milliseconds,0.000000
...,...,...,...
35783,0.000000e+00,% Linear Filtered,2015.671954
35784,0.000000e+00,% Anisotropic Filtered,2015.671955
35785,7.717031e+01,Avg Preemption Delay,2015.671955
35786,3.223443e+10,cpu-cycles,2015.683485


In [113]:
last_timestamp = df['t_s'].max()
recording_start_approx = last_timestamp - 30  # buffer for ~30s recording

# Look at data around that time
window = df[(df['t_s'] >= recording_start_approx - 15) & 
            (df['t_s'] <= recording_start_approx + 15)]
print(window[['t_s', 'counter_name', 'value']])

               t_s                     counter_name         value
412    1985.436051                  Clocks / Second  4.561761e+08
413    1985.436057                   GPU % Bus Busy  1.000000e+02
414    1985.436060             % Vertex Fetch Stall  0.000000e+00
415    1985.436062            % Texture Fetch Stall  0.000000e+00
416    1985.436065  L1 Texture Cache Miss Per Pixel -1.000000e+00
...            ...                              ...           ...
31470  2000.660165                       app_rss_mb  0.000000e+00
31471  2000.660166                       app_uss_mb  0.000000e+00
31472  2000.667832                       cpu-cycles  1.809695e+10
31473  2000.683080                       cpu-cycles  1.588675e+10
31474  2000.689103                       cpu-cycles  1.812310e+10

[31063 rows x 3 columns]


In [115]:
df['t_s'] = df['t_s'] - (window['t_s'].iloc[0])
df = df[df['t_s'] >= 0]
df

C:\Users\gkamt\AppData\Local\Temp\ipykernel_34272\3502949050.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['t_s'] = df['t_s'] - (window['t_s'].iloc[0])


,value,counter_name,t_s
412,4.561761e+08,Clocks / Second,0.000000
413,1.000000e+02,GPU % Bus Busy,0.000005
414,0.000000e+00,% Vertex Fetch Stall,0.000008
415,0.000000e+00,% Texture Fetch Stall,0.000011
416,-1.000000e+00,L1 Texture Cache Miss Per Pixel,0.000013
...,...,...,...
35783,0.000000e+00,% Linear Filtered,30.235903
35784,0.000000e+00,% Anisotropic Filtered,30.235903
35785,7.717031e+01,Avg Preemption Delay,30.235904
35786,3.223443e+10,cpu-cycles,30.247434


In [117]:
df["Time (s)"] = df["t_s"].round(2)
features_df = (df.pivot_table(
        index="Time (s)",
        columns="counter_name",
        values="value",
        aggfunc="mean", # if multiple rows have same counter_name and time bin, mean value is used
        observed=True)
        .sort_index())
features_df = features_df.reset_index()
features_df.columns.name = None
features_df

C:\Users\gkamt\AppData\Local\Temp\ipykernel_34272\783636871.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Time (s)"] = df["t_s"].round(2)


,Time (s),% Anisotropic Filtered,% Linear Filtered,% Nearest Filtered,% Non-Base Level Textures,% Prims Clipped,% Prims Trivially Rejected,% Shader ALU Capacity Utilized,% Shaders Busy,% Stalled on System Memory,...,face_tracking_overall_latency_milliseconds,face_tracking_processing_latency_milliseconds,gpu_frequency_mhz,gpu_level,gpu_util,guardian_gpu_ms,last_used_foveation_level,mem_frequency_mhz,screen_tears_per_second,timewarp_gpu_ms
0,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.01,0.0,0.0,0.0,0.0,1.182638,21.723063,0.0,0.0,0.086881,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.02,0.0,0.0,0.0,0.0,0.981906,27.797311,0.0,0.0,0.289989,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.03,0.0,0.0,0.0,0.0,0.329551,18.412748,0.0,0.0,0.001351,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.04,0.0,0.0,0.0,0.0,1.109824,14.619200,0.0,0.0,0.012935,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1935,30.21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1936,30.23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
1937,30.24,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1938,30.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [119]:
features_df = features_df.replace(0, np.nan)
features_df = features_df.dropna(axis=1, how='all')
features_df = features_df.ffill()
features_df = features_df.bfill()
features_df = features_df.dropna(axis=1, how='any')
features_df = features_df[features_df['Time (s)'] >= 7]
features_df

,Time (s),% Prims Clipped,% Prims Trivially Rejected,% Stalled on System Memory,% Texture L2 Miss,% Vertex Fetch Stall,ALU / Fragment,ALU / Vertex,Average Polygon Area,Average Vertices / Polygon,...,face_tracking_frame_to_frame_latency_milliseconds,face_tracking_frames_per_second,face_tracking_overall_latency_milliseconds,face_tracking_processing_latency_milliseconds,gpu_frequency_mhz,gpu_level,gpu_util,mem_frequency_mhz,screen_tears_per_second,timewarp_gpu_ms
243,7.00,2.265422,28.646619,0.42504,29.281914,0.695825,-1.0,-1.0,-1.0,0.878164,...,40.0,25.0,1000.0,1000.0,456.0,3.0,39.057395,2736.0,2.0,4.512586
244,7.01,2.265422,28.646619,0.42504,29.281914,0.695825,-1.0,-1.0,-1.0,0.878164,...,40.0,25.0,1000.0,1000.0,456.0,3.0,39.057395,2736.0,2.0,4.512586
245,7.02,2.265422,28.646619,0.42504,29.281914,0.695825,-1.0,-1.0,-1.0,0.878164,...,40.0,25.0,1000.0,1000.0,456.0,3.0,39.057395,2736.0,2.0,4.512586
246,7.03,2.265422,28.646619,0.42504,30.163250,0.629822,-1.0,-1.0,-1.0,0.878164,...,40.0,25.0,1000.0,1000.0,456.0,3.0,39.057395,2736.0,2.0,4.512586
247,7.04,2.265422,28.646619,0.42504,30.163250,0.629822,-1.0,-1.0,-1.0,0.878164,...,40.0,25.0,1000.0,1000.0,456.0,3.0,39.057395,2736.0,2.0,4.512586
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1935,30.21,2.265422,28.646619,0.42504,8.060784,0.294490,-1.0,-1.0,-1.0,0.878164,...,40.0,25.0,1000.0,1000.0,456.0,2.0,77.857423,2736.0,2.0,2.205620
1936,30.23,2.265422,28.646619,0.42504,8.060784,0.294490,-1.0,-1.0,-1.0,0.878164,...,40.0,25.0,1000.0,1000.0,456.0,2.0,77.857423,2736.0,2.0,2.205620
1937,30.24,2.265422,28.646619,0.42504,8.051286,0.422534,-1.0,-1.0,-1.0,0.878164,...,40.0,25.0,1000.0,1000.0,456.0,2.0,77.857423,2736.0,2.0,2.205620
1938,30.25,2.265422,28.646619,0.42504,8.051286,0.422534,-1.0,-1.0,-1.0,0.878164,...,40.0,25.0,1000.0,1000.0,456.0,2.0,77.857423,2736.0,2.0,2.205620


In [6]:
if features_df.shape[1] >= 50:
    features_df.to_csv("blind_1_cleaned.csv", index=False)